In [1]:
TEXT2SOURCEDICT = {}

In [54]:
import re
import pandas as pd
import os
def depersonalize(text):
    text = str(text)
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text= re.sub(url_regex, "url", text)
    text = re.sub("id[\d]*",'',text)
    text = re.sub("> ?",'',text)
    text = re.sub("@[\w]*",'',text)
#     text = re.sub("\+[\d]*", "", text)
    text = re.sub("[\d]+", "NUMBER", text)
    text = re.sub(">>", "", text)
#     text = re.sub("[\d]{3,100}", "", text)
    text = re.sub("[\t|\n|\r]", "", text)
    text = re.sub("[(OP)|url|nickname|phone_number]", "", text)
    return text

In [3]:
# tr test

In [12]:
def get_texts_from_folder(folder, txt, sep):
    txt_set = set()
    for f in os.listdir(folder):
        pth = os.path.join(folder, f)
        df_t = pd.read_csv(pth, sep=sep)
        texts_set = set(df_t[txt].tolist())
        txt_set = txt_set.union(texts_set)
    return txt_set

In [5]:
train_tes_texts_set = get_texts_from_folder('./gr0_train/',"INPUT:text", '\t')

In [6]:
fpaths = ['g1.tsv','g2.tsv','g3.tsv','g1_4.tsv','g2_4.tsv','g3_4.tsv','maybe_old/g1.tsv','maybe_old/g2.tsv']
for f in fpaths:
    pth = './directly_from_toloka/' + f
    df_c = pd.read_csv(pth, sep = '\t')
    golden_columns = [c for c in df_c.columns if 'GOLDEN' in c]
    df_c = df_c[~df_c[golden_columns[1]].isna()]
    train_tes_texts_set = train_tes_texts_set.union(set(df_c['INPUT:text'].tolist()))

In [7]:
def add_set_to_dict(text_set, category):
    for t in text_set:
        if t not in TEXT2SOURCEDICT:
            TEXT2SOURCEDICT[t] = category
            
        t_pr = depersonalize(t)
        if t_pr not in TEXT2SOURCEDICT:
            TEXT2SOURCEDICT[t_pr] = category

In [8]:
add_set_to_dict(train_tes_texts_set, "trtest")

In [9]:
len(TEXT2SOURCEDICT)

912

In [ ]:
# toloka

In [10]:
toloka_set = get_texts_from_folder('./gr1_toloka/','text',',')

In [13]:
add_set_to_dict(toloka_set, "toloka")

In [14]:
len(TEXT2SOURCEDICT)

12939

In [15]:
# "self"

In [16]:
df1_s = pd.read_csv('./gr2_self/export_ready_st1 - Sheet1.csv')
df2_s = pd.read_csv('./gr2_self/SELF_LABEL_export_ready_st2_multilabel_view.csv')
df2_s = df2_s[:1000]
df_self = pd.concat([df1_s,df2_s])

In [17]:
self_list = set(df_self['text'].tolist())
len(self_list)

2009

In [18]:
add_set_to_dict(self_list, "self")
len(TEXT2SOURCEDICT)

15356

In [19]:
# kw

In [20]:
df2_s = pd.read_csv('./gr2_self/SELF_LABEL_export_ready_st2_multilabel_view.csv')
df2_s = df2_s[1000:]
df3_s = pd.read_csv('./gr2_self/SELF_LABLE_export_ready_st3_multilabel_view.csv')
df4_s = pd.read_csv('./gr2_self/SELF_LABLE_export_ready_st4_multilabel_view.csv')
df_kw = pd.concat([df2_s,df3_s,df4_s])
kw_list = set(df_kw['text'].tolist())

In [21]:
add_set_to_dict(kw_list, "kw")
len(TEXT2SOURCEDICT)

51849

In [22]:
# "kwolga"

In [23]:
df_kwolga_list = []
folder = './gr3_kw_olga/'
for f in os.listdir(folder):
    filepath = os.path.join(folder, f)
    df_curr = pd.read_csv(filepath, header = None)
    df_kwolga_list.append(df_curr)
    
df_kwolga = pd.concat(df_kwolga_list)

In [24]:
kwolga_set = set(df_kwolga[0].tolist())

In [25]:
add_set_to_dict(kwolga_set, "kwolga")
len(TEXT2SOURCEDICT)

51887

In [26]:
# Считаем

In [27]:
from collections import Counter

In [28]:
df_sensitive_published = pd.read_csv("../sensitive_topics/sensitive_topics.csv")

In [29]:
len(df_sensitive_published)

33303

In [30]:
source_list = []
not_found = 0
for t in df_sensitive_published['text'].tolist():
    t_proc = depersonalize(t)
    if t in TEXT2SOURCEDICT:
        source_list.append(TEXT2SOURCEDICT[t])
    elif t_proc in TEXT2SOURCEDICT:
        source_list.append(TEXT2SOURCEDICT[t_proc])
    else:
        not_found += 1
        source_list.append('toloka')
print(" {} marked as toloka but actually not found".format(not_found))

 365 marked as toloka but actually not found


In [36]:
Counter(source_list).most_common(100)

[('kw', 21656), ('toloka', 9291), ('self', 1978), ('trtest', 378)]

In [37]:
train_tes_texts_set_proc = set([depersonalize(t) for t in train_tes_texts_set])

In [38]:
Counter(df_sensitive_published['text'].isin(train_tes_texts_set_proc))

Counter({False: 32925, True: 378})

In [39]:
df_sensitive_published['source'] = source_list

In [40]:
df_sensitive_published.to_csv("sensitive_topics_vs_source.csv", index = None)

# Восстанавливаем после переработки

In [30]:
df_sensitive_published = pd.read_csv("sensitive_topics_vs_source.csv")

In [31]:
len(df_sensitive_published)

33303

In [32]:
df_sensitive_published.head()

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,source
0,РРРРРРЯЯЯЯЯЯЯ РУССКИЕ ВАНЬКИ НИКОМУ НЕ НУЖНЫ!!...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,toloka
1,"По моему, быдло тот кто осуждает чужие предпоч...",0.0,0.0,0.0,0.0,1.0,0.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,toloka
2,Бабы это вообще безхуйные сиськастые ссущие си...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,trtest
3,изнасиловалиа говорила что девственницаЯ так д...,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,trtest
4,"и что ? не сосут только уебищные тни, видосов ...",0.0,0.0,0.0,0.0,1.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,toloka


# Присоединяем переразмеченную толоку

In [33]:
import pandas as pd

In [34]:
df_toloka_relabeled = pd.read_csv("TOLOKA_low_conf_for_manual_relabeling_ready.csv")

In [16]:
len(df_toloka_relabeled)

1788

In [35]:
df_toloka_relabeled_really_lbl = df_toloka_relabeled[~df_toloka_relabeled['vote1'].isna()]
# df_toloka_relabeled_no_really = df_toloka_relabeled[df_toloka_relabeled['vote1'].isna()]

In [36]:
df_sensitive_published_witout_dropped_toloka = df_sensitive_published[~df_sensitive_published['text'].isin(df_toloka_relabeled_really_lbl['text'])]

### Обрабатываем реально переразмеченную часть

In [37]:
df_toloka_relabeled_really_lbl = df_toloka_relabeled_really_lbl[['text','vote1','vote2','vote3']]

In [40]:
topics = list(df_sensitive_published.columns[1:-1])
len(topics)

18

In [41]:
collected_data = []
for i, el in df_toloka_relabeled_really_lbl.iterrows():
    assert type(el['vote1']) == str
    current_data = [el['text']]
    topics_list = [0] * len(topics)
    for vote in ['vote1','vote2','vote3']:
        if type(el[vote]) == str:
            voted_topic = el[vote]
            if voted_topic == 'none':
                break
            index_in_list = topics.index(voted_topic)
            topics_list[index_in_list] = 1
    current_data = current_data + topics_list + ['toloka_real_manual_relabel']
    collected_data.append(current_data)
    
df_toloka_relabeled_really_lbl_multiview = pd.DataFrame(data = collected_data, columns = df_sensitive_published.columns)

In [42]:
df_toloka_relabeled_really_lbl_multiview.sample(5)

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,source
398,"Удваиваю этого поехавшего. Нет места лучше, че...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,toloka_real_manual_relabel
242,"Рссылкой,считаю,придёться замарачиваться тпк ч...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,toloka_real_manual_relabel
305,АХУЕТЬ! да в Лесото просто жируют! ещё и гандж...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,toloka_real_manual_relabel
152,Хочу купитькартыдолларовой или рублевой?В не ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,toloka_real_manual_relabel
168,"Иди дрочи на губастых тощих шлюх, пидор.",0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,toloka_real_manual_relabel


### ту толоку которую не переразмечаем - НЕ ТРОГАЕМ 

In [43]:
refactored_sensitive_df = pd.concat([df_sensitive_published_witout_dropped_toloka,
                                    df_toloka_relabeled_really_lbl_multiview,
                                    ])
refactored_sensitive_df.drop_duplicates(subset = ['text'],inplace = True)

In [85]:
# Sanity check

In [44]:
len(df_sensitive_published)

33303

In [45]:
len(refactored_sensitive_df)

33304

In [46]:
original_set = set(df_sensitive_published['text'])
refubrished_set = set(refactored_sensitive_df['text'])

In [47]:
len(original_set & refubrished_set)

33303

# Добавляем семплы из специфических источников но без ключевых слов (все размечены реально руками)

In [48]:
df_no_kw = pd.read_csv("no_keyword_spec_forums_for_self_label_ready.csv")

In [49]:
df_no_kw.drop(columns = ['topic'], inplace = True)

In [50]:
df_no_kw.head()

,processed,vote1,vote2
0,Удлинители бывают различных видов: металличес...,none,NaN
1,1 степень - способность к общению со снижением...,none,NaN
2,Просто получается что любви то у тебя и нет и...,none,NaN
3,Ну если ты за жрал гавно и кормил комаров то ...,social_injustice,NaN
4,... или я уже ничего не понимаю.,none,NaN


In [51]:
collected_data = []
for i, el in df_no_kw.iterrows():
    assert type(el['vote1']) == str
    current_data = [el['processed']]
    topics_list = [0] * len(topics)
    for vote in ['vote1','vote2']:
        if type(el[vote]) == str:
            voted_topic = el[vote]
            if voted_topic == 'none':
                break
            index_in_list = topics.index(voted_topic)
            topics_list[index_in_list] = 1
    current_data = current_data + topics_list + ['no_kw_manual_relabel']
    collected_data.append(current_data)
    
df_no_kw_multiview = pd.DataFrame(data = collected_data, columns = df_sensitive_published.columns)

In [52]:
df_no_kw_multiview.sample(3)

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,source
416,Мракобесие рождается от непонимания тех или ин...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,no_kw_manual_relabel
584,"— Ну да, пока купит, пока подключит — будешь н...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,no_kw_manual_relabel
109,"Этот вопрос вы можете задать скоро, кажется ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,no_kw_manual_relabel


In [55]:
df_no_kw_multiview['text'] = df_no_kw_multiview['text'].apply(depersonalize)

In [56]:
df_no_kw_multiview.sample(3)

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,source
414,"Разве можно требоватьневозможное?"" это говорит...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,no_kw_manual_relabel
500,", с Днём рождения! Здоровья и счастья",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,no_kw_manual_relabel
379,Человек-это заключительный этап эволюции...от ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,no_kw_manual_relabel


In [57]:
len(refactored_sensitive_df)

33304

In [58]:
df_sensitive_new = pd.concat([refactored_sensitive_df, df_no_kw_multiview])

In [59]:
len(df_sensitive_new)

33904

In [61]:
from collections import Counter
Counter(df_sensitive_new['source']).most_common(10)

[('kw', 21656),
 ('toloka', 8766),
 ('self', 1978),
 ('no_kw_manual_relabel', 600),
 ('toloka_real_manual_relabel', 526),
 ('trtest', 378)]

In [62]:
df_sensitive_new.to_csv("sensitive_topics_vers3.csv",index = None)